In [2]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [3]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [7]:
######################
## D. Leer los jsons
######################

# 1. Leer los jsons y guardarlos en un dataframe
paths = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt'), '.json')
df_renacyt = pd.concat([pd.read_json(path) for path in paths], axis=0)

df_renacyt.to_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))
df_renacyt.astype(str).to_excel(os.path.join(os.getcwd(),'outputs','excels','renacyt.xlsx'), index=False)

# 2. Generar lista de urls a partir de ['ctiVitae']
urls = []
cti_vitaes = df_renacyt['ctiVitae'].tolist()
for cit_vitae in cti_vitaes:
    url = f'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador={cit_vitae}'
    urls.append(url)

urls = list(set(urls))
print(f'Se generaron {len(urls)} urls')

urls = urls[0:10]
urls

Se generaron 7152 urls


['https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=26379',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=59272',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=174796',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=68188',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=69972',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=29289',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=30696',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=2221',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=4820',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=58750']

In [5]:
######################
## E. Ejecutar
######################

i = 0
list_ids = []
# for index, row in df_boletas.head(100).iterrows(): # Pruebas
for index, row in df_boletas.iterrows(): # Correr todo
    try:
        # Obtener el id y la url
        id = '"id":"' + row['id'] + '"'
        url = row['url']

        # Cargar la url
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "text-3xl-600")))

        # Guardar los id para buscar en json
        list_ids.append(id)
        print(f'{str(i).zfill(2)} -> url: {url}')
        i += 1
    except:
        break

['https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=5093',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=223284',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=171899',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=125717',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=123317',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=75377',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=69493',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=55629',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=37145',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=13555',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=3726',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=1993',
 'https://servicio-renacyt.

In [3]:
######################
## D. Ejecución
######################

# 0. Definir los datos de la cuenta
url = 'https://servicio-renacyt.concytec.gob.pe/busqueda-de-investigadores/'

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iniciar har
start_har(proxy, 'renacyt')

# 3. Abrir la página
driver.get(url)

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver


In [4]:
# 4. Obtener el .har y guardar los json
time.sleep(TIME_SLEEP*2)
har = proxy.har

In [5]:
i = 0
for entrie in har.get('log').get('entries'):
    aux_entrie = str(entrie)

    if '"codigoRegistro"' in aux_entrie and '"institucionLaboralPrincipal"' in aux_entrie:
        string_content = entrie.get('response').get('content').get('text')
        data_list = json.loads(string_content)
        with open(os.path.join(os.getcwd(),'jsons','renacyt','renacyt_' + str(i).zfill(2) + '_data.json'), 'w') as json_file:
            json.dump(data_list, json_file, indent=4)
        i += 1

In [6]:
stop_chrome_driver(driver, server, proxy)

In [7]:
# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
